In [1]:
!pip install gast==0.2.2

In [20]:
!pip install tensorflow==2.0.0-alpha0 

     |████████████████████████████████| 79.9MB 45.1MB/s eta 0:00:01   |███▏                            | 8.0MB 3.2MB/s eta 0:00:23     |████▏                           | 10.5MB 3.2MB/s eta 0:00:22     |█████████████████████▋          | 54.0MB 45.1MB/s eta 0:00:01
     |████████████████████████████████| 3.0MB 25.1MB/s eta 0:00:01     |██▉                             | 266kB 25.1MB/s eta 0:00:01
     |████████████████████████████████| 419kB 53.1MB/s eta 0:00:01
ERROR: tensorflow-serving-api 1.14.0 has requirement tensorflow~=1.14.0, but you'll have tensorflow 2.0.0a0 which is incompatible.
  Found existing installation: tb-nightly 1.14.0a20190603
    Uninstalling tb-nightly-1.14.0a20190603:
      Successfully uninstalled tb-nightly-1.14.0a20190603
  Found existing installation: tf-estimator-nightly 1.14.0.dev2019060501
    Uninstalling tf-estimator-nightly-1.14.0.dev2019060501:
      Successfully uninstalled tf-estimator-nightly-1.14.0.dev2019060501
  Found existing installation: tensorf

In [3]:
!pip install keras-self-attention

In [21]:
import json
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.losses import sparse_categorical_crossentropy # used for integer targets
from tensorflow.keras.optimizers import Nadam
from random import shuffle, seed, sample
from tqdm import tqdm, tqdm_notebook
import numpy as np
import os
#from keras_self_attention import SeqSelfAttention

Finding all titles, training, and target sets composed by Chopin

## Parse Input file into title, train, target

In [5]:
titles_h = []
train_h = []
target_h = []
with open('data/Haydn_input.json', 'r') as handle: # EDIT WITH COMPOSER
    for i,line in enumerate(tqdm(handle)):
        song = json.loads(line)
        titles_h.append(song['title'])
        train_h.append(song['train'])
        target_h.append(song['target'])

8it [00:04,  1.82it/s]


KeyboardInterrupt: 

In [5]:
titles = []
train = []
target = []
with open('data/Brahms_input.json', 'r') as handle: # EDIT WITH COMPOSER
    for i,line in enumerate(tqdm(handle)):
        song = json.loads(line)
        titles.append(song['title'])
        train.append(song['train'])
        target.append(song['target'])

26it [00:28,  1.09s/it]


In [8]:
len(set(titles))

271

## Below for Parsing Chopin

In [41]:
unique_chopin = []
keep_indices = []
for i,song in enumerate(titles):
    song = song.lower()
    if 'andante' in song:
        song = 'andante op 22'
    if 'ballade no. 2' in song:
        song = 'ballade no 2'
    if 'barcarolle' in song:
        song = 'barcarolle'
    if 'berceuse' in song:
        song = 'barceuse'
    if 'etude op. 10 no. 1' in song:
        song = 'etude op. 10 no. 1'
    if 'etude op. 10 no. 12' in song:
        song = 'etude op. 10 no. 12'
    if 'etude op. 10 no. 2' in song:
        song = 'etude op. 10 no. 2'
    if 'etude op. 10 no. 4' in song:
        song = 'etude op. 10 no. 4'
    if 'etude op. 10 no. 8' in song:
        song = 'etude op. 10 no. 8'
    if 'etude op. 25 no. 1' in song:
        song = 'etude op. 25 no. 1'
    if 'etude op. 25 no. 10' in song:
        song = 'etude op. 25 no. 10'
    if 'etude op. 25 no. 11' in song:
        song = 'etude op. 25 no. 11'
    if 'etude op. 25 no. 12' in song:
        song = 'etude op. 25 no. 12'
    if 'etude op. 25 no. 6' in song:
        song = 'etude op. 25 no. 6'
    if 'op. 49' in song:
        song = 'fantasy op. 49'
    if 'polonaise-fantasie' in song:
        song = 'polonaise-fantasie'
    if 'scherzo no. 2' in song:
        song = 'scherzo no. 2'
    if 'scherzo no. 3' in song:
        song = 'scherzo no. 3'
    if 'scherzo no. 4' in song:
        song = 'scherzo no. 4'
    if 'sonata no. 3' in song:
        song = 'sonata no. 3'
    if song not in unique_chopin:
        unique_chopin.append(song)
        keep_indices.append(i)

## Below for Parsing Brahms

In [6]:
unique_brahms = []
keep_indices = []
for i,song in enumerate(titles):
    song = song.lower()
    if 'sonata in' in song:
        song = 'sonata in f minor op 5'
    if song not in unique_brahms:
        unique_brahms.append(song)
        keep_indices.append(i)

## Below for Parsing Sonatas

In [3]:
unique_son = []
keep_indices = []
for i,song in enumerate(titles):
    if song not in unique_son:
        unique_son.append(song)
        keep_indices.append(i)

## Below for Parsing Hadyn

In [4]:
unique_ha = []
keep_indices = []
for i,song in enumerate(titles_h):
    if song not in unique_ha:
        unique_ha.append(song)
        keep_indices.append(i)

## Keeping Unique Songs

In [7]:
unique_train = []
for i,each in enumerate(train):
    if i in keep_indices:
        unique_train.append(each)

In [8]:
unique_target = []
for i, each in enumerate(target):
    if i in keep_indices:
        unique_target.append(each)

In [9]:
len(unique_train)

23

In [10]:
len(unique_target)

23

In [11]:
len(unique_brahms)

23

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K

## Self Attention

In [23]:
class SeqSelfAttention(tf.keras.layers.Layer):

    ATTENTION_TYPE_ADD = 'additive'
    ATTENTION_TYPE_MUL = 'multiplicative'

    def __init__(self,
                 units=32,
                 attention_width=None,
                 attention_type=ATTENTION_TYPE_ADD,
                 return_attention=False,
                 history_only=False,
                 kernel_initializer='glorot_normal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 use_additive_bias=True,
                 use_attention_bias=True,
                 attention_activation=None,
                 attention_regularizer_weight=0.0,
                 **kwargs):
        """Layer initialization.
        For additive attention, see: https://arxiv.org/pdf/1806.01264.pdf
        :param units: The dimension of the vectors that used to calculate the attention weights.
        :param attention_width: The width of local attention.
        :param attention_type: 'additive' or 'multiplicative'.
        :param return_attention: Whether to return the attention weights for visualization.
        :param history_only: Only use historical pieces of data.
        :param kernel_initializer: The initializer for weight matrices.
        :param bias_initializer: The initializer for biases.
        :param kernel_regularizer: The regularization for weight matrices.
        :param bias_regularizer: The regularization for biases.
        :param kernel_constraint: The constraint for weight matrices.
        :param bias_constraint: The constraint for biases.
        :param use_additive_bias: Whether to use bias while calculating the relevance of inputs features
                                  in additive mode.
        :param use_attention_bias: Whether to use bias while calculating the weights of attention.
        :param attention_activation: The activation used for calculating the weights of attention.
        :param attention_regularizer_weight: The weights of attention regularizer.
        :param kwargs: Parameters for parent class.
        """
        self.supports_masking = True
        self.units = units
        self.attention_width = attention_width
        self.attention_type = attention_type
        self.return_attention = return_attention
        self.history_only = history_only
        if history_only and attention_width is None:
            self.attention_width = int(1e9)

        self.use_additive_bias = use_additive_bias
        self.use_attention_bias = use_attention_bias
        self.kernel_initializer = tf.keras.initializers.get(kernel_initializer)
        self.bias_initializer = tf.keras.initializers.get(bias_initializer)
        self.kernel_regularizer = tf.keras.regularizers.get(kernel_regularizer)
        self.bias_regularizer = tf.keras.regularizers.get(bias_regularizer)
        self.kernel_constraint = tf.keras.constraints.get(kernel_constraint)
        self.bias_constraint = tf.keras.constraints.get(bias_constraint)
        self.attention_activation = tf.keras.activations.get(attention_activation)
        self.attention_regularizer_weight = attention_regularizer_weight
        self._backend = tf.keras.backend.backend()

        if attention_type == SeqSelfAttention.ATTENTION_TYPE_ADD:
            self.Wx, self.Wt, self.bh = None, None, None
            self.Wa, self.ba = None, None
        elif attention_type == SeqSelfAttention.ATTENTION_TYPE_MUL:
            self.Wa, self.ba = None, None
        else:
            raise NotImplementedError('No implementation for attention type : ' + attention_type)

        super(SeqSelfAttention, self).__init__(**kwargs)

    def get_config(self):
        config = {
            'units': self.units,
            'attention_width': self.attention_width,
            'attention_type': self.attention_type,
            'return_attention': self.return_attention,
            'history_only': self.history_only,
            'use_additive_bias': self.use_additive_bias,
            'use_attention_bias': self.use_attention_bias,
            'kernel_initializer': tf.keras.regularizers.serialize(self.kernel_initializer),
            'bias_initializer': tf.keras.regularizers.serialize(self.bias_initializer),
            'kernel_regularizer': tf.keras.regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer': tf.keras.regularizers.serialize(self.bias_regularizer),
            'kernel_constraint': tf.keras.constraints.serialize(self.kernel_constraint),
            'bias_constraint': tf.keras.constraints.serialize(self.bias_constraint),
            'attention_activation': tf.keras.activations.serialize(self.attention_activation),
            'attention_regularizer_weight': self.attention_regularizer_weight,
        }
        base_config = super(SeqSelfAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def build(self, input_shape):
        if isinstance(input_shape, list):
            input_shape = input_shape[0]
        if self.attention_type == SeqSelfAttention.ATTENTION_TYPE_ADD:
            self._build_additive_attention(input_shape)
        elif self.attention_type == SeqSelfAttention.ATTENTION_TYPE_MUL:
            self._build_multiplicative_attention(input_shape)
        super(SeqSelfAttention, self).build(input_shape)

    def _build_additive_attention(self, input_shape):
        feature_dim = input_shape[2]

        self.Wt = self.add_weight(shape=(feature_dim, self.units),
                                  name='{}_Add_Wt'.format(self.name),
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)
        self.Wx = self.add_weight(shape=(feature_dim, self.units),
                                  name='{}_Add_Wx'.format(self.name),
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)
        if self.use_additive_bias:
            self.bh = self.add_weight(shape=(self.units,),
                                      name='{}_Add_bh'.format(self.name),
                                      initializer=self.bias_initializer,
                                      regularizer=self.bias_regularizer,
                                      constraint=self.bias_constraint)

        self.Wa = self.add_weight(shape=(self.units, 1),
                                  name='{}_Add_Wa'.format(self.name),
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)
        if self.use_attention_bias:
            self.ba = self.add_weight(shape=(1,),
                                      name='{}_Add_ba'.format(self.name),
                                      initializer=self.bias_initializer,
                                      regularizer=self.bias_regularizer,
                                      constraint=self.bias_constraint)

    def _build_multiplicative_attention(self, input_shape):
        feature_dim = input_shape[2]

        self.Wa = self.add_weight(shape=(feature_dim, feature_dim),
                                  name='{}_Mul_Wa'.format(self.name),
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)
        if self.use_attention_bias:
            self.ba = self.add_weight(shape=(1,),
                                      name='{}_Mul_ba'.format(self.name),
                                      initializer=self.bias_initializer,
                                      regularizer=self.bias_regularizer,
                                      constraint=self.bias_constraint)

    def call(self, inputs, mask=None, **kwargs):
        if isinstance(inputs, list):
            inputs, positions = inputs
            positions = K.cast(positions, 'int32')
            mask = mask[1]
        else:
            positions = None

        input_len = K.shape(inputs)[1]

        if self.attention_type == SeqSelfAttention.ATTENTION_TYPE_ADD:
            e = self._call_additive_emission(inputs)
        elif self.attention_type == SeqSelfAttention.ATTENTION_TYPE_MUL:
            e = self._call_multiplicative_emission(inputs)

        if self.attention_activation is not None:
            e = self.attention_activation(e)
        e = K.exp(e - K.max(e, axis=-1, keepdims=True))
        if self.attention_width is not None:
            ones = tf.ones((input_len, input_len))
            if self.history_only:
                local = tf.linalg.band_part(
                    ones,
                    K.minimum(input_len, self.attention_width - 1),
                    0,
                )
            else:
                local = tf.linalg.band_part(
                    ones,
                    K.minimum(input_len, self.attention_width // 2),
                    K.minimum(input_len, (self.attention_width - 1) // 2),
                )
            e = e * K.expand_dims(local, 0)
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            mask = K.expand_dims(mask)
            e = K.permute_dimensions(K.permute_dimensions(e * mask, (0, 2, 1)) * mask, (0, 2, 1))

        # a_{t} = \text{softmax}(e_t)
        s = K.sum(e, axis=-1)
        s = K.tile(K.expand_dims(s, axis=-1), K.stack([1, 1, input_len]))
        a = e / (s + K.epsilon())

        # l_t = \sum_{t'} a_{t, t'} x_{t'}
        v = K.batch_dot(a, inputs)
        if self.attention_regularizer_weight > 0.0:
            self.add_loss(self._attention_regularizer(a))

        if positions is not None:
            pos_num = K.shape(positions)[1]
            batch_indices = K.tile(K.expand_dims(K.arange(K.shape(inputs)[0]), axis=-1), K.stack([1, pos_num]))
            pos_indices = K.stack([batch_indices, positions], axis=-1)
            v = tf.gather_nd(v, pos_indices)
            a = tf.gather_nd(a, pos_indices)

        if self.return_attention:
            return [v, a]
        return v

    def _call_additive_emission(self, inputs):
        input_shape = K.shape(inputs)
        batch_size, input_len = input_shape[0], input_shape[1]

        # h_{t, t'} = \tanh(x_t^T W_t + x_{t'}^T W_x + b_h)
        q, k = K.dot(inputs, self.Wt), K.dot(inputs, self.Wx)
        q = K.tile(K.expand_dims(q, 2), K.stack([1, 1, input_len, 1]))
        k = K.tile(K.expand_dims(k, 1), K.stack([1, input_len, 1, 1]))
        if self.use_additive_bias:
            h = K.tanh(q + k + self.bh)
        else:
            h = K.tanh(q + k)

        # e_{t, t'} = W_a h_{t, t'} + b_a
        if self.use_attention_bias:
            e = K.reshape(K.dot(h, self.Wa) + self.ba, (batch_size, input_len, input_len))
        else:
            e = K.reshape(K.dot(h, self.Wa), (batch_size, input_len, input_len))
        return e

    def _call_multiplicative_emission(self, inputs):
        # e_{t, t'} = x_t^T W_a x_{t'} + b_a
        e = K.batch_dot(K.dot(inputs, self.Wa), K.permute_dimensions(inputs, (0, 2, 1)))
        if self.use_attention_bias:
            e = e + self.ba
        return e

    def compute_output_shape(self, input_shape):
        if isinstance(input_shape, list):
            input_shape, pos_shape = input_shape
            output_shape = (input_shape[0], pos_shape[1], input_shape[2])
        else:
            output_shape = input_shape
        if self.return_attention:
            attention_shape = (input_shape[0], output_shape[1], input_shape[1])
            return [output_shape, attention_shape]
        return output_shape

    def compute_mask(self, inputs, mask=None):
        if isinstance(inputs, list):
            mask = mask[1]
        if self.return_attention:
            return [mask, None]
        return mask

    def _attention_regularizer(self, attention):
        batch_size = K.cast(K.shape(attention)[0], K.floatx())
        input_len = K.shape(attention)[-1]
        return self.attention_regularizer_weight * K.sum(K.square(K.batch_dot(
            attention,
            K.permute_dimensions(attention, (0, 2, 1))) - tf.eye(input_len))) / batch_size

    @staticmethod
    def get_custom_objects():
        return {'SeqSelfAttention': SeqSelfAttention}

## Modeling

In [24]:
def create_model(seq_len, unique_notes, dropout=0.3, output_emb=100, rnn_unit=128, dense_unit=64):
    inputs = tf.keras.layers.Input(shape=(seq_len,))
    embedding = tf.keras.layers.Embedding(input_dim=unique_notes, output_dim=output_emb, input_length=seq_len)(inputs)
    forward_pass = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(rnn_unit, return_sequences=True))(embedding)
    forward_pass , att_vector = SeqSelfAttention(
        return_attention=True,
        attention_activation='sigmoid',
        attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
        attention_width=50,
        kernel_regularizer=tf.keras.regularizers.l2(1e-4),
        bias_regularizer=tf.keras.regularizers.l1(1e-4),
        attention_regularizer_weight=1e-4,
        )(forward_pass)
    forward_pass = tf.keras.layers.Dropout(dropout)(forward_pass)
    forward_pass = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(rnn_unit, return_sequences=True))(forward_pass)
    forward_pass , att_vector2 = SeqSelfAttention(
        return_attention=True,
        attention_activation='sigmoid',
        attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, # multiplicative
        attention_width=50,
        kernel_regularizer=tf.keras.regularizers.l2(1e-4),
        bias_regularizer=tf.keras.regularizers.l1(1e-4),
        attention_regularizer_weight=1e-4,
        )(forward_pass)
    forward_pass = tf.keras.layers.Dropout(dropout)(forward_pass)
    forward_pass = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(rnn_unit))(forward_pass)
    forward_pass = tf.keras.layers.Dropout(dropout)(forward_pass)
    forward_pass = tf.keras.layers.Dense(dense_unit)(forward_pass)
    forward_pass = tf.keras.layers.LeakyReLU()(forward_pass)
    outputs = tf.keras.layers.Dense(unique_notes, activation = "softmax")(forward_pass)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='generate_scores_rnn')
    return model

In [25]:
with open('data/Brahms_index.json', 'r') as read_file:
    dict_index = json.load(read_file)

In [26]:
unique_notes = len(dict_index)

In [27]:
unique_notes

62921

In [28]:
model = create_model(50, unique_notes)

In [29]:
model.summary()

Model: "generate_scores_rnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 50, 100)           6292100   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 50, 256)           176640    
_________________________________________________________________
seq_self_attention (SeqSelfA [(None, 50, 256), (None,  65537     
_________________________________________________________________
dropout (Dropout)            (None, 50, 256)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 50, 256)           296448    
_________________________________________________________________
seq_self_attention_1 (SeqSel [(None, 50, 256), 

In [30]:
optimizer = Nadam()

checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 model=model)
checkpoint_dir = 'models/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
loss_fn = sparse_categorical_crossentropy

In [38]:
class TrainModel:
  
    def __init__(self, epochs, sample_train, sample_target, batch_nnet_size, batch_song, optimizer, checkpoint, loss_fn,
               checkpoint_prefix, total_songs, model):
        self.epochs = epochs
        self.sample_train = sample_train
        self.sample_target = sample_target
        self.batch_nnet_size = batch_nnet_size
        self.batch_song = batch_song
        self.optimizer = optimizer
        self.checkpoint = checkpoint
        self.loss_fn = loss_fn
        self.checkpoint_prefix = checkpoint_prefix
        self.total_songs = total_songs
        self.model = model
    
    def train(self):
        for epoch in tqdm.notebook.tqdm(range(self.epochs),desc='epochs'):
            # for each epochs, we shuffle the list of all the datasets
            c = list(zip(self.sample_train, self.sample_target))
            shuffle(c)
            self.sample_train, self.sample_target = zip(*c)
            loss_total = 0
            steps = 0
            steps_nnet = 0

            # Iterate all songs by the length of sample input (total_songs) and batches (batch_song)
            for i in tqdm.notebook.tqdm(range(0,self.total_songs, self.batch_song), desc='MUSIC'):
                # EXAMPLE: [0,5,10,15,20] FOR TOTAL_SONGS = 20 AND BATCH_SONG = 5
                steps += 1
                #inputs_nnet_large, outputs_nnet_large = generate_batch_song(
                 #   self.sample_input, self.batch_song, start_index=i, fs=self.frame_per_second, 
                  #  seq_len=seq_len, use_tqdm=False) # We use the function that have been defined here
                #inputs_nnet_large = np.array(self.note_tokenizer.transform(inputs_nnet_large), dtype=np.int32)
                #outputs_nnet_large = np.array(self.note_tokenizer.transform(outputs_nnet_large), dtype=np.int32)
                
                # EXAMPLE LARGE INPUTS = ARRAY([1,2,3,4],[2,3,4,5],[2,3,4,5],[2,3,4,5],[1,2,3,4])
                input_batch = [y for x in self.sample_train[i:i+self.batch_song] for y in x]
                output_batch = [y for x in self.sample_target[i:i+self.batch_song] for y in x]
                c = list(zip(input_batch, output_batch))
                all_sample = sample(c, 10000)
#                 start = c[0:5000]
#                 middle = int(len(c)/2)
#                 middle_left = c[middle:middle-5000]
#                 middle_right = c[middle:middle+5000]
#                 end = c[-5000:]
#                 all_sample = start + middle_left + middle_right + end
                input_batch, output_batch = zip(*all_sample)
                inputs_nnet_large = np.array(input_batch)
                outputs_nnet_large = np.array(output_batch)

                # Get an index of all windows in a song
                index_shuffled = np.arange(start=0, stop=len(inputs_nnet_large))
                np.random.shuffle(index_shuffled)
                
                for nnet_steps in tqdm.notebook.tqdm(range(0,len(index_shuffled),self.batch_nnet_size)):
                    steps_nnet += 1
                    current_index = index_shuffled[nnet_steps:nnet_steps+self.batch_nnet_size]

                    inputs_nnet, outputs_nnet = inputs_nnet_large[current_index], outputs_nnet_large[current_index]

                    # To make sure no exception thrown by tensorflow on autograph
                    if len(inputs_nnet) // self.batch_nnet_size != 1:
                        break
                    #print(outputs_nnet)
                    loss = self.train_step(inputs_nnet, outputs_nnet)
                    loss_total += tf.math.reduce_sum(loss)
                    if steps_nnet % 20 == 0:
                        print("epochs {} | Steps {} | total loss : {}".format(epoch + 1, steps_nnet,loss_total))

                    #checkpoint.save(file_prefix = self.checkpoint_prefix)

    @tf.function
    def train_step(self, inputs, targets):
        with tf.GradientTape() as tape:
            prediction = self.model(inputs)
            loss = self.loss_fn(targets, prediction)
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        return loss

In [39]:
seq_len = 50
EPOCHS = 5
BATCH_SONG = 5
BATCH_NNET_SIZE = 96
TOTAL_SONGS = len(unique_target)
FRAME_PER_SECOND = 5

In [40]:
train_class = TrainModel(EPOCHS, unique_train, unique_target, BATCH_NNET_SIZE, BATCH_SONG, optimizer, checkpoint, loss_fn, checkpoint_prefix, 
                        TOTAL_SONGS, model)
train_class.train()

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:56: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


epochs 1 | Steps 20 | total loss : 15205.609375
epochs 1 | Steps 40 | total loss : 30384.552734375
epochs 1 | Steps 60 | total loss : 44479.734375
epochs 1 | Steps 80 | total loss : 58393.34765625
epochs 1 | Steps 100 | total loss : 72149.6796875


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:56: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epochs 5 | Steps 20 | total loss : 11807.72265625
epochs 5 | Steps 40 | total loss : 23443.177734375
epochs 5 | Steps 60 | total loss : 34737.0
epochs 5 | Steps 80 | total loss : 46278.89453125
epochs 5 | Steps 100 | total loss : 57675.03125


epochs 5 | Steps 120 | total loss : 68344.6953125
epochs 5 | Steps 140 | total loss : 79353.6796875
epochs 5 | Steps 160 | total loss : 89990.0390625
epochs 5 | Steps 180 | total loss : 100513.0546875
epochs 5 | Steps 200 | total loss : 111343.203125


epochs 5 | Steps 220 | total loss : 122093.484375
epochs 5 | Steps 240 | total loss : 133083.8125
epochs 5 | Steps 260 | total loss : 144258.546875
epochs 5 | Steps 280 | total loss : 154561.3125
epochs 5 | Steps 300 | total loss : 165478.125


epochs 5 | Steps 320 | total loss : 176586.125
epochs 5 | Steps 340 | total loss : 189231.734375
epochs 5 | Steps 360 | total loss : 201295.03125
epochs 5 | Steps 380 | total loss : 213440.640625
epochs 5 | Steps 400 | total loss : 225432.796875


epochs 5 | Steps 440 | total loss : 248830.296875
epochs 5 | Steps 460 | total loss : 260317.1875
epochs 5 | Steps 480 | total loss : 271686.09375
epochs 5 | Steps 500 | total loss : 282689.59375
epochs 5 | Steps 520 | total loss : 293851.03125




In [41]:
model.save('brahms_seq.h5')